In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical



# Loading the Dataset

df_data = pd.read_csv("winequality-red.csv")


In [ ]:
df_data.tail()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5
1598,6.0,0.310,0.47,3.6,0.067,18.0,42.0,0.99549,3.39,0.66,11.0,6


In [ ]:
df_data.shape

(1599, 12)

In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [ ]:
df_data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [ ]:
df_dataX = df_data.drop(columns=["quality"]).values  # Features
df_dataY = df_data["quality"].values  # Target



In [ ]:
scaler = MinMaxScaler()
X_sc = scaler.fit_transform(df_dataX)


In [ ]:
classification = True  # Set True for classification, False for regression

if classification:
    # Classification: One-hot encode target
    y1 = to_categorical(df_dataY - df_dataY.min())  # Ensure class indices start at 0

else:
    # Regression: Reshape target to (samples, 1)
    y1 = df_dataY.reshape(-1, 1)



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sc, y1, test_size=0.2, random_state=42)

# Step 6: Reshape input for RNN
X_train = np.expand_dims(X_train, axis=1)  # Add timestep dimension
X_test = np.expand_dims(X_test, axis=1)


In [ ]:
output_units = y_train.shape[1] if classification else 1
activation = 'softmax' if classification else 'linear'
loss = 'categorical_crossentropy' if classification else 'mse'

model = Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),  # Input shape: (timesteps, features)
    LSTM(64, return_sequences=False),  # Produce a single output at the last timestep
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(output_units, activation=activation)  # Output layer matches target shape
])

In [ ]:
model.compile(
    optimizer='adam',
    loss=loss,
    metrics=['accuracy'] if classification else ['mae']
)

# Step 8: Train the model
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32)


Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.2921 - loss: 1.7617 - val_accuracy: 0.4023 - val_loss: 1.6093
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4540 - loss: 1.5502 - val_accuracy: 0.4023 - val_loss: 1.3200
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4254 - loss: 1.3241 - val_accuracy: 0.4023 - val_loss: 1.2029
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4154 - loss: 1.2792 - val_accuracy: 0.4023 - val_loss: 1.1691
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4333 - loss: 1.2100 - val_accuracy: 0.4023 - val_loss: 1.1516
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4566 - loss: 1.1570 - val_accuracy: 0.4062 - val_loss: 1.1377
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4589 - loss: 1.1600 - val_accuracy: 0.5898 - val_loss: 1.1138
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5208 - loss: 1.1632 - val_accuracy: 0.4688 - val_loss

In [ ]:
# Step 9: Evaluate the model
results = model.evaluate(X_test, y_test)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5721 - loss: 0.9508 


In [ ]:
loss, metric = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Metric: {metric}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5721 - loss: 0.9508 
Test Loss: 0.9594404101371765, Test Metric: 0.559374988079071
